# VGG16 Model for Project 3 
### By Tianle Zhu

In [7]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
import time
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.externals import joblib
from sklearn.ensemble import GradientBoostingClassifier 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer, Input
from keras.layers import Conv2D, Dense, Flatten, Dropout, Activation
from tensorflow.keras.layers import BatchNormalization, Reshape, MaxPooling2D, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.applications import VGG16
import tensorflow as tf
from tensorflow import keras
from keras import Model
import os
from shutil import copyfile, move
from tqdm import tqdm
import h5py
from sklearn import metrics
#from tensorflow.keras import optimizers
from keras import optimizers

In [2]:
batch_size = 16

In [3]:
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    horizontal_flip=True,
    vertical_flip=True)

train_data_dir = "../data/train_set/data/train"
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    shuffle=True,
    target_size=(64, 64),
    batch_size=batch_size,
    class_mode='binary')

validation_datagen = ImageDataGenerator(rescale=1. / 255)
validation_data_dir = "../data/train_set/data/validation"
validation_generator = validation_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(64, 64),
    batch_size=batch_size,
    class_mode='binary')

input_shape = (64,64,3)
num_classes = 2

Found 2384 images belonging to 2 classes.
Found 616 images belonging to 2 classes.


In [16]:
vgg = VGG16(input_shape = input_shape, weights = 'imagenet', include_top = False)
for layer in vgg.layers:
    layer.trainable = False

x = Flatten()(vgg.output)
x = Dense(128, activation = 'relu')(x)   # we can add a new fully connected layer but it will increase the execution time.
x = Dense(1, activation = 'sigmoid')(x)  # adding the output layer with sigmoid function 

model = Model(inputs = vgg.input, outputs = x)
# learning rate try  0.01, 0.001, 0.001
#Adam = keras.optimizers.Adam(lr = 0.001)
Adam = optimizers.Adam(lr = 0.05)
model.compile(loss = 'binary_crossentropy', optimizer = Adam, metrics = ['accuracy'])

In [17]:
start_time_train = time.time()
history = model.fit_generator(train_generator,
                   steps_per_epoch = 38,  # this should be equal to total number of images in training set. Change this for better results. 
                   epochs = 10,  # change this for better results
                   class_weight = [85,1], # change this for better results
                   validation_data = validation_generator,
                   validation_steps = 10)
elapsed_time_train = time.time() - start_time_train

Epoch 1/10
38/38 [==============================] - 18s 482ms/step - loss: 2.7634 - acc: 0.8240 - val_loss: 2.3914 - val_acc: 0.8500
Epoch 2/10
38/38 [==============================] - 23s 595ms/step - loss: 3.5398 - acc: 0.7780 - val_loss: 2.9892 - val_acc: 0.8125
Epoch 3/10
38/38 [==============================] - 21s 549ms/step - loss: 3.1727 - acc: 0.8010 - val_loss: 3.1885 - val_acc: 0.8000
Epoch 4/10
38/38 [==============================] - 20s 526ms/step - loss: 3.0416 - acc: 0.8092 - val_loss: 2.7270 - val_acc: 0.8289
Epoch 5/10
38/38 [==============================] - 22s 570ms/step - loss: 3.1203 - acc: 0.8043 - val_loss: 2.3914 - val_acc: 0.8500
Epoch 6/10
38/38 [==============================] - 21s 553ms/step - loss: 3.5398 - acc: 0.7780 - val_loss: 2.4910 - val_acc: 0.8438
Epoch 7/10
38/38 [==============================] - 18s 464ms/step - loss: 3.7234 - acc: 0.7664 - val_loss: 3.2881 - val_acc: 0.7937
Epoch 8/10
38/38 [==============================] - 16s 415ms/step - 

In [18]:
train_acc = history.history['acc'][-1]
validation_acc = history.history['val_acc'][-1]

In [19]:
validation_generator.reset()

In [20]:
start_time_test = time.time()
pred = model.predict_generator(validation_generator,verbose = 1, steps= 616/16)
elapsed_time_test = time.time() - start_time_test

39/38 [==============================] - 12s 305ms/step


In [21]:
model.evaluate_generator(validation_generator,steps = 616/16)

[2.976257020776924, 0.8133116883116883]

In [22]:
# Calculate AUC
fpr, tpr, thresholds = metrics.roc_curve(validation_generator.classes, pred)
AUC_vaule = metrics.auc(fpr, tpr)
AUC_vaule

0.5

In [23]:
print("Training time:", elapsed_time_train)
print("Testing time:", elapsed_time_test)
print("Training Accuracy:", train_acc)
print("Validation Accuracy:", validation_acc)
print("AUC for VGG16 moldel:", AUC_vaule)

Training time: 186.29671001434326
Testing time: 11.886746883392334
Training Accuracy: 0.8125
Validation Accuracy: 0.85
AUC for VGG16 moldel: 0.5


In [32]:
# testing_datagen = ImageDataGenerator(rescale=1. / 255)
# testing_data_dir = "../data/train_set/data/test"

# testing_generator = testing_datagen.flow_from_directory(
#     testing_data_dir,
#     target_size=(64, 64),
#     batch_size=batch_size,
#     class_mode='binary')

Found 8 images belonging to 1 classes.
